# LAPORAN RULE BASE INFORMATION EXTRACTION

**NAMA: ZANUAR RIKZA ADITIYA**    
**NIM: 230411100087**  

### PROSES MENDOWNLOAD PDF DARI WEBSITE PUTUSAN

In [ ]:
import csv
import requests

def download_files_from_csv(csv_file_path, output_folder):
    with open(csv_file_path, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        for row in csv_reader:
                
            url = row[20]  # assuming the URL is in the first column
            if url != "":
                download_file(url, output_folder)
       

def download_file(url, output_folder):
    response = requests.get(url)
    if response.status_code == 200:
        file_name = url.split("/")[-1]
        file_path = f"{output_folder}/{file_name}.pdf"
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {url}")

if __name__ == "__main__":
    csv_file_path = "./data/metaPidanaUmum.csv"  # Ubah path sesuai dengan lokasi file CSV Anda
    output_folder = "./pdf"  # Ubah path sesuai dengan lokasi folder output Anda
    download_files_from_csv(csv_file_path, output_folder)


## PROSES MERUBAH PDF MENJADI TEXT

In [91]:
import os
from pypdf import PdfReader


# Mendapatkan list seluruh file pdf dari folder yang ditentukan disini folder pdf
dir_path = os.listdir("./pdf/")

# Mengiterasi list file pdf yang didapatkan
for filename in dir_path:
    # Membaca fuke odf debgab libraru pypdf
    reader = PdfReader( "./pdf/" + filename)
    
    # mengambil semua pages sebagai list yang dapat di iterasi
    pages = reader.pages
    
    # membuat file jika tidak ada atau merewrite file jika ada berdasarkan nama file pdf
    with open(f"./INPUT/{filename.split(".")[0]}.txt", "w", encoding="utf-8") as file:
        for page in pages:
            # menulis text setiap page ke file yang ditentuakan
            file.writelines(page.extract_text())

## PROSES MENGEKSTRAKSI INFORMASI DARI TEXT YANG DI DAPATKAN DARI PDF

Dikarenakan text yang saya extract dari pdf format nya berbeda dengan yang ada pada contoh INPUT, jadi saya merubah banyak code yang ada pada GenerateEntity

#### FUNGSI MENCARI NOMOR PUTUTSAN

In [105]:
# karena nomor putusan ditext rata rata di awali dengan string Nomor
# dan diakhiri dengan string DEMI saya menggunakan pattern ini untuk mengambil informasi nomor putusan

def cari_nomor_putusan(data):
    # Mendapatkan index dari string nomor di data text nya untuk text awal yang diambil
    start_index = data.find("Nomor")
    
    # Mendapatkan index dari string DEMI di data text nya untuk index akhir
    end_index = data.find("DEMI")
    
    # start_index ditambah panjang Nomor karena untuk exclude kan string Nomor dan mengambil hanya nomor putusan nya saja
    return data[start_index + len("Nomor"):end_index].strip()

### FUNGSI MENCARI TERDAKWA

In [106]:
import os

# Proses Mengextract data disin ada dua fungsi
# fungsi pertama untuk menangani ektrasi berdasarkan finder atau pattern karena 
# informasi terdakwa ada dua pattern untuk hanya 1 terdakwa dan lebih dari satu

def exctract_terdakwa(data, finder):
    # Mencari index pattern awalan yang ada pada data untuk mencari terdakwa
    
    # menyimpan pattern yang mungkin di list
    pattern = [
        "perkara t erdakwa",
        "perkara"
    ]
    start = data.find(finder)    

    # jika start tidak ditemukan maka cari dengan pattern lain
    for i in pattern:
        if start != -1:
            break
        
        start = data.find(i)
    
 
    data = data[start:]
    # proses mengeklude kan nama lengkap agar yang diambil hanya nama nya saja
    nama_raw = data[data.find("nama lengkap") + len("nama lengkap : "):]
    nama_raw = nama_raw[:data.find(";")] + " "
    nama_raw = nama_raw[:nama_raw.find("tempat")] + " "
    nama_raw = " ".join(nama_raw[:nama_raw.find("2")].replace(";", "").split())
    return nama_raw.strip()

def cari_terdakwa(data):
    # Menghapus enter dan mengganti nya dengan spasi, lalu merubah data menjadi lowercase semua
    data = " ".join(data.split("\n")).lower()
    
    # mencatata total terdakwa
    total_terdakwa = 0
    
    # list terdakwa
    list_terdakwa = []
    
    # menghitung terdakwa
    while data.find("terdakwa " + str(total_terdakwa + 1)) != -1:
        total_terdakwa += 1
        
    # menangani terdakwa jika kurang dari 1 dan lebih dari satu
    if total_terdakwa < 1:
        # jika cuma satu maka menggunakan pattern perkara terdakwa
        list_terdakwa.append(exctract_terdakwa(data, "perkara terdakwa"))
    else:
        # jika lebih dari satu menggunakan pattern terdakwa i
        for i in range(1, total_terdakwa + 1):
            list_terdakwa.append((exctract_terdakwa(data, "terdakwa " + str(i))))
    
    return list_terdakwa
            

### FUNSI MENACARI TUNTUTAN TINDAK PIDANA

In [107]:
# Membuat fungsi mencari tuntutan
# dengan pattern yang di awali kata melakukan dan di akhiri sebagaimana atau sesuai

def cari_tuntutan_tp(data):

    # menghapus enter dan spasi berlebih
    data = " ".join(data.lower().split())

    # mengambil text setelah melakaukan di tambah spasi agar jika hasil dari index akhir -1 huruf tidak terhapus
    data = data[data.find("melakukan"):] + " "
    
    # mengambi text yang berada pada sebelum text sebagaimana jika ada
    data = data[:data.find("sebagaimana")] + " "
    # mengambi text yang berada pada sebelum text sebagaimana sesuai jika ada
    data = data[:data.find("sesuai")] + " "
    
    # membersihkan text dari kutip dan spasi berlebih
    return " ".join(data.split()[3:]).replace("“", "").replace("”", "").replace("\"", "").strip()

### FUNGSI MENCARI KUHP TUNTUTAN

In [108]:
# Membuat fungsi mencari KUHP TUNTUTAN

def cari_kuhp_tp(data):
    # Menghapus enter dan merubah ke huruf kecil
    data = " ".join(data.split()).lower()
    
    # Mencari start text
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    
    # Menggambil start - akhirt
    data = data[start:]
    
    # Kolesi Pattern
    pattern = [
        "pasal",
        "sebagaimana diatur dan diancam dalam",
    ]
    
    start = -1
    
    # Mengiterasi pattern 
    for pat in pattern:
        if data.find(pat) != -1:
            # Memotong motong text sesuai pattern yang di tentukan
            data = data[data.find(pat):] + " "
            data = data[:data.find("2.")] + " "
            data = data[:data.find(";")] + " "
            data = data[:data.find("tentang")] + " "
            data = data.strip()
            break

    
    return data

### FUNGSI MENCARI TUNTUTAN HUKUMAN

In [109]:
# FUNGSI MENCARI TUNTUTAN HUKUM

def cari_hukuman_tp(data):
    # Menghapus enter dan merubah ke huruf kecil

    data = " ".join(data.split()).lower()
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    data = data[start:]
    
    # Memotong motong text sesuai pattern yang di tentukan
    start = data.find("selama")
    end = data.find("3. ")
    hasil = data[start:end] + " "
    hasil = hasil[:hasil.find(";")]
        
    return hasil

### FUNGSI MENCARI PUTUTSAN TINDAKAN PIDANA

In [110]:
# FUNGSI MENCARI PUTUSAN TP

def cari_putusan_tp(data):
    # Menghapus enter dan merubah ke huruf kecil

    data = " ".join(data.split())
    # Memulai pencarian setelah kata MENGADILI:
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else (data.find("M E N G A D I L I:") if data.find("M E N G A D I L I:") != -1 else data.find("M E N G A D I L I :"))
    data = data[start:]
    end = data.find("2.")
    data = data[:end]
    
    data = " ".join(data.lower().split())

    # Memotong motong text sesuai pattern
    data = data[data.find("melakukan"):] + " "
    data = data[:data.find("sebagaimana")] + " "
    data = data[:data.find(" s ebagaimana")] + " "
    data = data[:data.find("sesuai")] + " "
    data = data.strip()
    
    # Membersihkan text dari kutip dan spasi
    data = " ".join(data.split()[3:]).replace("“", "").replace("”", "").replace("\"", "").strip()
    
    # jika panjang data sama dengan 0 berarti terdakwa tidak terbukti bersalah
    if len(data) == 0:
        return "tidak terbukti bersalah"    
    
    return data

### FUNGSI HUKUMAN PUTUSAN

In [111]:
# FUNGSI MENCARI PUTUSAN HUKUMAN

def cari_hukuman_p(data):
    # Menghapus enter dan merubah ke huruf kecil
    data = " ".join(data.split())
    # Memulai pencarian setelah kata MENGADI
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else  (data.find("M E N G A D I L I:") if data.find("M E N G A D I L I:") != -1 else data.find("M E N G A D I L I :"))
    
    # Memotong motong text sesuai pattern
    data = data[start:]
    start = data.find("selama")
    end = data.find("3. ")
    hasil = data[start:end] + " "
    hasil = hasil[:hasil.find(";")].strip()
    
    # jika panjang data sama dengan 0 berarti terdakwa dibebaskan
    if len(hasil) == 0:
        return "dibebaskan"
        
    return hasil

### FUNGSI MENCARI HAKIM

In [154]:
# FUNGSI MENCARI HAKIM

def cari_hakim(data):
    
    # Menghapus enter dan merubah ke huruf kecil
    data = " ".join(data.split())
    # Memulai pencarian setelah kata MENGADILI
    mengadili = [
        "MENGADILI:",
        "M E N G A D I L I:",
        "M E N G A D I L I :",
        "M E N G A D I L I"
    ]
    start = -1
    for i in mengadili:
        if start != -1:
            break
      
        start = data.find(i)
    
    data = data[start:].lower()
    data = data[data.find("6."):]
    data = data[data.find("oleh kami") + len("oleh kami") if data.find("oleh kami") != -1 else data.find("oleh") + len("oleh"):].strip()
   

    data_hakim = {
        
    }
    
    # Hakim ketua
    data_hakim["hakim ketua"] = data[:data.find("sebagai hakim")].strip()
    
    if data_hakim["hakim ketua"][-1] == ",":
        data_hakim["hakim ketua"] = data_hakim["hakim ketua"][:-1]
    
    if data_hakim["hakim ketua"][0] == ",":
        data_hakim["hakim ketua"] = data_hakim["hakim ketua"][1:].strip()
        
    # Menampung hakim anggota dengan di dalam list dengan pattern tertentu
    hakim_anggota = data[data.find("sebagai hakim ketua") + len("sebagai hakim ketua"):]
    hakim_anggota = hakim_anggota[:hakim_anggota.find("masing")].strip().split("dan")
    
    
    # Membersihkan hakim anggota
    for hakim in range(len(hakim_anggota)):
        if hakim_anggota[hakim][0] == ",":
            # Membersihkan hakim dari space berlebih dan , di awal
            hakim_anggota[hakim] = hakim_anggota[hakim][1:].strip()
        
        if hakim_anggota[hakim][-1] == ",":
            # Membersihkan hakim dari space berlebih dan , di akhir
            hakim_anggota[hakim] = hakim_anggota[hakim][:-1].strip()
        
            
    data_hakim["hakim anggota"] = hakim_anggota
    return data_hakim



### FUNGSI MENCARI TANGGAL PUTUSAN

In [155]:
def cari_tanggal_putusan(data):
    # Menghapus enter dan merubah ke huruf kecil

    data = " ".join(data.split())
    
    # Memulai pencarian setelah kata MENGADI
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else  (data.find("M E N G A D I L I:") if data.find("M E N G A D I L I:") != -1 else data.find("M E N G A D I L I :"))
    data = data[start:]
    
    # Memotong motong text sesuai pattern tanggal
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("tanggal") + len("tanggal"):]
    
    # Membersihkan text
    data = data[:data.find(",")].strip()
    return data

### FUNGSI MENCARI PANITERA

In [156]:
def cari_panitera(data):
    
    # Menghapus enter dan merubah ke huruf kecil
    data = " ".join(data.split())
    
    # Memulai pencarian setelah kata MENGADILI
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else  (data.find("M E N G A D I L I:") if data.find("M E N G A D I L I:") != -1 else data.find("M E N G A D I L I :"))
    data = data[start:]
    
    # Mengambil text setalh point 6.
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    
    # Memotong motong text sesuai pattern panitera
    data = data[data.find("dibantu oleh") + len("dibantu oleh "):]
    data = data[:data.find("panitera")].strip()
    return data

### FUNGSI MENCARI PENUNTUT

In [157]:
def cari_penuntut(data):
    
    # Menghapus enter dan merubah ke huruf kecil
    data = " ".join(data.split())
    
    # Memulai pencarian setelah kata MENGADI
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else  (data.find("M E N G A D I L I:") if data.find("M E N G A D I L I:") != -1 else data.find("M E N G A D I L I :"))
    data = data[start:]
    
    # Mengambil text setalh point 6.
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    
    # Memotong motong text sesuai pattern panitera
    data = data[data.find("dihadiri oleh") + len("dihadiri oleh "):]
    data = data[:data.find("penuntut")].strip()
    
    return data

## ENTITI GENERATOR

In [158]:
# DI FUNGSI generateEntity hanya melakukan penggabungan fungsi fungsi dan menampilkan 
# hasil lalu di tulis dalam file output


def generateEntity(fileName):
    input_folder = "./INPUT/"
    output_folder = "./OUTPUT/"
    
    with open(output_folder + "O-" + fileName, "w", encoding="utf-8") as output_file:

        with open(input_folder + fileName, "r", encoding="utf-8") as input_file:
            # MEMANGGIL SEMUA FUNGSI
            data = input_file.read()
            nomor_putusan = cari_nomor_putusan(data)
            terdakwa = cari_terdakwa(data)
            
            tuntutan_tp = cari_tuntutan_tp(data)
            kuhp = cari_kuhp_tp(data)
            hukuman = cari_hukuman_tp(data)
            
            putusan_tp = cari_putusan_tp(data)
            hukuman_p = cari_hukuman_p(data)
            tanggal = cari_tanggal_putusan(data)
            hakim = cari_hakim(data)
            panitera = cari_panitera(data)
            penuntut = cari_penuntut(data)

            
            # MENAMPIKAN DAN MENULIS SELURUH HASIL KE FILE TXT
            print("Nomor Putusan: " + nomor_putusan)
            output_file.writelines("Nomor Putusan: " + nomor_putusan + "\n")
            for terdakwa_index in range(len(terdakwa)):
                print("Terdakwa Ke: " + str(terdakwa_index + 1))
                output_file.writelines("Terdakwa Ke: " + str(terdakwa_index + 1) + "\n")
                print("Nama Terdakwa: " + terdakwa[terdakwa_index])
                output_file.writelines("Nama Terdakwa: " + terdakwa[terdakwa_index] + "\n" + "\n")
                print()

            print("TUNTUTAN")
            output_file.writelines("TUNTUTAN" + "\n")
            print("Tindak Pidana: " + tuntutan_tp )
            output_file.writelines("Tindak Pidana: " + tuntutan_tp + "\n")
            print("Melanggar KUHP: " + kuhp)
            output_file.writelines("Melanggar KUHP: " + kuhp + "\n")
            print("Tuntutan Pidana: " + hukuman)
            output_file.writelines("Tuntutan Pidana: " + hukuman + "\n" + "\n")
            
            print()
            print("PUTUSAN")
            output_file.writelines("PUTUSAN" + "\n")
            print("Tindak Pidana: " + putusan_tp )
            output_file.writelines("Tindak Pidana: " + putusan_tp + "\n")
            print("Hukunan Pidana: " + hukuman_p)
            output_file.writelines("Hukunan Pidana: " + hukuman_p + "\n")
            print("Tanggal Putusan: " + tanggal)
            output_file.writelines("Tanggal Putusan: " + tanggal + "\n"  + "\n")
            
            print()
            print("Hakim Ketua: " + hakim["hakim ketua"])
            output_file.writelines("Hakim Ketua: " + hakim["hakim ketua"] + "\n")
            
            for h in range(len(hakim["hakim anggota"])):
                print("Hakim Anggota " + str(h + 1) + ": ", hakim["hakim anggota"][h])
                output_file.writelines("Hakim Anggota " + str(h + 1) + ": " + hakim["hakim anggota"][h] + "\n")
                
            print("Panitera: " + panitera)
            output_file.writelines("Panitera: " + panitera + "\n")
            print("Penuntut Umum: " + penuntut)
            output_file.writelines("Penuntut Umum: " + penuntut + "\n")

In [159]:
import os


# FUNGSI GERBANG MASUK SEMUA KODE 
def main():

    start_path = "./INPUT/"
    daftarFile = os.listdir(start_path)

    for tiapFile in daftarFile:
        print("======= ROW HASIL =======")    
        hasil = generateEntity(tiapFile)
        print("======= ROW AKHIR =======")
        print()
 

main()


======= ROW HASIL =======
Nomor Putusan: XX/Pid.Sus/2024/PN Pkl
Terdakwa Ke: 1
Nama Terdakwa: terdakwa

TUNTUTAN
Tindak Pidana: membawa pergi seorang wanita yang belum dewasa, tanpa dikehendaki orang tuanya atau walinya tetapi dengan persetujuannya
Melanggar KUHP: pasal 83 jo. 76 f undang-undang nomor 35 tahun 2014
Tuntutan Pidana: selama 1 (satu) tahun dan 6 (enam) bulan dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan

PUTUSAN
Tindak Pidana: membawa pergi seorang wanita yang belum dewasa
Hukunan Pidana: selama 1 (satu) tahun
Tanggal Putusan: 18 maret 2024

Hakim Ketua: agus maksum mulyohadi, s.h., m.h.
Hakim Anggota 1:  nofan hidayat, s.h., m.h.
Hakim Anggota 2:   budi setyawan, s.h.
Panitera: siroju munir, s.h., m.h.,
Penuntut Umum: eko hertanto, s.h.,m.h.,
======= ROW AKHIR =======

======= ROW HASIL =======
Nomor Putusan: XXX/Pid.Sus/2023/PN Pkl
Terdakwa Ke: 1
Nama Terdakwa: xxxxxx

TUNTUTAN
Tindak Pidana: dengan sengaja melakukan